# process origin dataset

## load origin data

In [1]:
import os

In [2]:
def summary_pro(texts):
    text = ""
    for t in texts:
        text = text + t.strip('\n') + ' '
    return text.strip()

data_name_list = ['train', 'val', 'test']

data_dict = {
    "train": [],
    "val": [],
    "test": []
}

for data_name in data_name_list:
    path = os.path.join('/data/ectsum', data_name)
    path = os.path.join(path, 'gt_summaries')
    files = os.listdir(path)
    for file in files:
        file_path = os.path.join(path, file)
        with open(file_path, 'r') as f:
            texts = f.readlines()
            data_dict[data_name].append(summary_pro(texts))
            f.close()

In [ ]:
len(data_dict["train"])

## split sentence

In [4]:
# * We don't need to use spacy since the pattern is very simple.
for data_name in data_name_list:
    tmp = []
    for para in data_dict[data_name]:
        sentence_list = para.split('. ')
        sentence_list = [s + "." for s in sentence_list[:-1]] + [sentence_list[-1]]
        tmp += sentence_list
    data_dict[data_name] = tmp

In [ ]:
len(data_dict["train"])

## create huggingface datasets type data

In [ ]:
import datasets

In [ ]:
dataset = datasets.DatasetDict({
    "train": datasets.Dataset.from_dict({"text": data_dict["train"]}),
    "valid": datasets.Dataset.from_dict({"text": data_dict["val"]}),
    "test": datasets.Dataset.from_dict({"text": data_dict["test"]}),
})

dataset

In [ ]:
union_dataset = datasets.concatenate_datasets([dataset["train"], dataset["valid"], dataset["test"]])

union_dataset

In [9]:
union_dataset.save_to_disk("./tmp/ectsum")

# process new dataset

## get embedding & save

### sup-simcse-bert-base-uncased

In [ ]:
!torchrun --nproc_per_node=8 ../embedding/sup-simcse-bert-base-uncased.py \
    --input_dataset "./tmp/ectsum" \
    --output_dataset "your_output_dir" \
    --train_size 6981 \
    --valid_size 1009 \
    --test_size 2001 \

### e5-large-v2

In [ ]:
!torchrun --nproc_per_node=8 ../embedding/e5-large-v2.py \
    --input_dataset "./tmp/ectsum" \
    --output_dataset "your_output_dir" \
    --train_size 6981 \
    --valid_size 1009 \
    --test_size 2001 \

### bge-large-en

In [ ]:
!torchrun --nproc_per_node=8 ../embedding/bge-large-en.py \
    --input_dataset "./tmp/ectsum" \
    --output_dataset "your_output_dir" \
    --train_size 6981 \
    --valid_size 1009 \
    --test_size 2001 \